<a href="https://colab.research.google.com/github/R3gm/stablepy/blob/main/stablepy_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stablepy

Install dependencies

In [ ]:
!pip install stablepy==0.4.1 -q

To use the version with the latest changes, you can install directly from the repository.

`pip install -q git+https://github.com/R3gm/stablepy.git`

Download our models and other stuffs

In [ ]:
%cd /content/

# Model
!wget https://huggingface.co/frankjoshua/toonyou_beta6/resolve/main/toonyou_beta6.safetensors
!wget https://huggingface.co/RunDiffusion/Juggernaut-XL-v9/resolve/main/Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors

# VAE
!wget https://huggingface.co/fp16-guy/anything_kl-f8-anime2_vae-ft-mse-840000-ema-pruned_blessed_clearvae_fp16_cleaned/resolve/main/anything_fp16.safetensors

# LoRAs
!wget https://civitai.com/api/download/models/183149 --content-disposition
!wget https://civitai.com/api/download/models/97655 --content-disposition

# Embeddings
!wget https://huggingface.co/embed/negative/resolve/main/bad-hands-5.pt
!wget https://huggingface.co/embed/negative/resolve/main/bad-artist.pt

# Upscaler
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth

# Inference with Stable diffusion 1.5

First, we pass the path of the model we will use.

The default task is txt2img but it can be changed to other tasks like canny

In [ ]:
from stablepy import Model_Diffusers
import torch

model_path = "./toonyou_beta6.safetensors"
vae_path = "./anything_fp16.safetensors"

model = Model_Diffusers(
    base_model_id = model_path, # path to the model
    task_name = "canny", # task
    vae_model = vae_path, # path vae
)

You can see the different tasks that can be used with sd1.5 with the following code:

In [ ]:
from stablepy import SD15_TASKS

SD15_TASKS

To switch tasks or models, we can call `model.load_pipe()` and specify the new task or model. This will load the necessary components.

In [ ]:
model.load_pipe(
    base_model_id = model_path, # path to the model
    task_name = "txt2img", # task
    vae_model = None, # Use default VAE
)

Simple generation using a sampler and a specified prompt weight.

In [ ]:
images, image_list = model(
    prompt = "cat, (masterpiece), (best quality)",
    sampler="DPM++ SDE Karras",
    syntax_weights="Classic-original",
)

images[0]

The different samplers that can be used can be checked in the following way:

In [ ]:
from stablepy import scheduler_names

scheduler_names

Prompt weight is the syntax and method used to emphasize certain parts of the prompt. If you want to get results similar to other popular implementations, you can use "Classic-original" with a SD1.5 model.

In [ ]:
from stablepy import ALL_PROMPT_WEIGHT_OPTIONS

ALL_PROMPT_WEIGHT_OPTIONS

We will use a basic txt2img task in which we can specify different common parameters, such as Loras, embeddings, upscaler, etc.

In [ ]:
from IPython.display import display

lora1_path = "./EmptyEyes_Diffuser_v10.safetensors"
lora2_path = "./FE_V2.safetensors" # pixel art lora
upscaler_path = "./RealESRGAN_x4plus.pth"

images, image_list = model(
    prompt = "pixel art (masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,",
    negative_prompt = "(worst quality, low quality, letterboxed), bad_artist_token, bad_hand_token",
    img_width = 513,
    img_height = 1024,
    num_images = 1,
    num_steps = 30,
    guidance_scale = 8.0,
    clip_skip = True, # Clip skip to the penultimate layer, in other implementations it is equivalent to use clipskip 2.
    seed = -1, # random seed
    sampler="DPM++ SDE Karras",
    syntax_weights="Classic-original",

    lora_A = lora1_path,
    lora_scale_A = 0.8,
    lora_B = lora2_path,
    lora_scale_B = 0.9,

    textual_inversion=[("bad_artist_token", "./bad-artist.pt"), ("bad_hand_token", "./bad-hands-5.pt")], # Is a list of tuples with [("<token_activation>","<path_embeding>"),...]

    upscaler_model_path = upscaler_path, # Upscale the image and Hires-fix
    upscaler_increases_size=1.5,
    hires_steps = 25,
    hires_denoising_strength = 0.35,
    hires_prompt = "", # If this is left as is, the main prompt will be used instead.
    hires_negative_prompt = "",
    hires_sampler = "Use same sampler",

    #By default, the generated images are saved in the current location within the 'images' folder.
    image_storage_location = "./images",

    #You can disable saving the images with this parameter.
    save_generated_images = False,
)

for image in images:
  display(image)

## ControlNet

In [ ]:
model.load_pipe(
    base_model_id = model_path,
    task_name = "canny",
    # Use default VAE
)

Our control image will be this one, to which the processor will apply Canny, and then use ControlNet to generate the final image.

In [ ]:
from PIL import Image
import os

def download_image(img_url):
    filename = os.path.basename(img_url)
    !wget -q {img_url} -O {filename}
    img = Image.open(filename)
    img.thumbnail((300, 300))
    display(img)
    return filename  # return the path

control_image = download_image("https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/images/bird.png")

Inference with canny

In [ ]:
images, image_list = model(
    prompt = "(masterpiece, best quality), bird",
    negative_prompt = "(worst quality, low quality, letterboxed)",
    image = control_image,
    preprocessor_name = "Canny", # Needed to activate the Canny preprocessor
    preprocess_resolution = 512, # It is the resize of the image that will be obtained from the preprocessor.
    image_resolution = 768, # The equivalent resolution to be used for inference.
    controlnet_conditioning_scale = 1.0, # ControlNet Output Scaling in UNet
    control_guidance_start = 0.0, # ControlNet Start Threshold (%)
    control_guidance_end= 1.0, # ControlNet Stop Threshold (%)

    upscaler_model_path = upscaler_path,
    upscaler_increases_size=1.3,

    # By default, 'hires-fix' is applied when we use an upscaler; to deactivate it, we can set 'hires steps' to 0
    hires_steps = 0,
)

for image in images:
  display(image)

Valid `preprocessor_name` depending on the task:


| Task name    | Preprocessor Name |
|----------|-------------------|
|canny|"None" "Canny"|
|mlsd|"None" "MLSD"|
| openpose | "None" "Openpose" |
|scribble|"None" "HED" "Pidinet"|
|softedge|"None" "HED" "Pidinet" "HED safe" "Pidinet safe"|
|segmentation|"None" "UPerNet"|
|depth|"None" "DPT" "Midas"|
|normalbae|"None" "NormalBae"|
|lineart|"None" "Lineart" "Lineart coarse" "None (anime)" "LineartAnime"|
|shuffle|"None" "ContentShuffle"|
|ip2p||
|pattern||


## Adetailer

In [ ]:
model.load_pipe(
    base_model_id = model_path,
    task_name = "txt2img",
)

There must be a match of parameters for good results to be obtained with adetailer, it is also useful to use `strength` in adetailer_inpaint_params with low values ​​below 0.4.

In [ ]:
# These are the parameters that adetailer A uses by default, but we can modify them if needed, the same applies to adetailer B.
adetailer_params_A = {
    "face_detector_ad" : True,
    "person_detector_ad" : True,
    "hand_detector_ad" : False,
    "prompt": "", # The main prompt will be used if left empty
    "negative_prompt" : "",
    "strength" : 0.35, # need low values
    "mask_dilation" : 4,
    "mask_blur" : 4,
    "mask_padding" : 32,
    "inpaint_only" : True, # better
    "sampler" : "Use same sampler",
}

images, image_list = model(
    prompt = "(masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,",
    negative_prompt = "(worst quality, low quality, letterboxed)",
    img_width = 512,
    img_height = 1024,
    num_images = 1,
    num_steps = 30,
    guidance_scale = 8.0,
    clip_skip = True,
    seed = 33,
    sampler="DPM++ SDE Karras",

    FreeU=True, # Improves diffusion model sample quality at no costs.
    adetailer_A=True,
    adetailer_A_params=adetailer_params_A,

    adetailer_B=True, # "If we don't use adetailer_B_params, it will use default values.

    # By default, the upscaler will be deactivated if we don't pass a model to it.
    # It's also valid to use a url to the model, Lanczos or Nearest.
    #upscaler_model_path = "Lanczos",
)

for image in images:
  display(image)

## Inpaint

In [ ]:
model.load_pipe(
    base_model_id = model_path,
    task_name = "inpaint",
)

We can specify the directory of our mask image, but we can also generate it, which is what we'll do in this example

You need a mouse to draw on this canvas.

In [ ]:
images, image_list = model(
    image = control_image,
    # image_mask = "/my_mask.png",
    prompt = "a blue bird",
    strength = 0.5,
    negative_prompt = "(worst quality, low quality, letterboxed)",
    image_resolution = 768,  # The equivalent resolution to be used for inference.
    sampler="DPM++ SDE Karras",
)

for image in images:
  display(image)

If you're using a device without a mouse or Jupyter Notebook outside of Colab, the function to create a mask automatically won't work correctly. Therefore, you'll need to specify the path of your mask image manually.

# Styles
These are additions to the prompt and negative prompt to utilize a specific style in generation. By default, there are only 9 of these, and we can know their names by using:

In [ ]:
model.STYLE_NAMES

But if we want to use other styles, we can load them through a JSON, like this one for example.
Here are more JSON style files: [PromptStylers](https://github.com/wolfden/ComfyUi_PromptStylers), [sdxl_prompt_styler](https://github.com/ali1234/sdxl_prompt_styler/tree/main)

In [ ]:
!wget https://raw.githubusercontent.com/ahgsql/StyleSelectorXL/main/sdxl_styles.json

In [ ]:
model.load_style_file("sdxl_styles.json")

The file was loaded with 77 styles replacing the previous ones, now we can see the new names:

In [ ]:
model.STYLE_NAMES

Now we can use the style in the inference.

In [ ]:
# Image to Image task.
model.load_pipe(
    base_model_id = model_path,
    task_name = "img2img",
)

# We can also use multiple styles in a list ["Silhouette", "Kirigami"]
images, image_list = model(
    style_prompt = "Silhouette", # The style will be added to the prompt and negative prompt
    image = control_image,
    prompt = "a bird",
    negative_prompt = "worst quality",
    strength = 0.48,
    image_resolution = 512,
    sampler="DPM++ SDE Karras",
)

for image in images:
  display(image)

#Verbosity Level
To change the verbosity level, you can use the logger from StablePy


In [ ]:
import logging
from stablepy import logger

logging_level_mapping = {
    'DEBUG': logging.DEBUG,
    'INFO': logging.INFO,
    'WARNING': logging.WARNING,
    'ERROR': logging.ERROR,
    'CRITICAL': logging.CRITICAL
}

Verbosity_Level = "WARNING" # Messages INFO and DEBUG will not be printed

logger.setLevel(logging_level_mapping.get(Verbosity_Level, logging.INFO))

# LCM and TCD

Latent Consistency Models (LCM) can generate images in a few steps. When selecting the 'LCM Auto-Loader' or 'TCD Auto-Loader' sampler, the model automatically loads the LCM_LoRA for the task. Generally, guidance_scale is used at 1.0 or a maximum of 2.0, with steps between 4 and 8.



In [ ]:
# Generating an image with txt2img
model.load_pipe(
    base_model_id = model_path,
    task_name = "txt2img",
)
images, image_list = model(
    prompt = "(masterpiece, best quality), 1girl, collarbone, wavy hair, looking at viewer, blurry foreground, upper body, necklace, contemporary, plain pants, ((intricate, print, pattern)), ponytail, freckles, red hair, dappled sunlight, smile, happy,",
    negative_prompt = "(worst quality, low quality, letterboxed)",
    num_images = 1,
    num_steps = 7,
    guidance_scale = 1.0,
    sampler="LCM Auto-Loader",  # or 'TCD Auto-Loader'
    syntax_weights="Classic", # (word:weight) and (word) for prompts weights
    disable_progress_bar = True,
    save_generated_images = False,
    display_images = True,
)

# Using the image generated in img2img
# If we use the same model and VAE, we can switch tasks quickly
model.load_pipe(
    base_model_id = model_path,
    task_name = "img2img",
)
images_i2i, image_list = model(
    prompt = "masterpiece, sunlight",
    image = images[0], # only one image
    style_prompt = "Disco", # Apply a style
    strength = 0.70,
    num_steps = 5,
    guidance_scale = 1.0,
    sampler="LCM Auto-Loader",
    disable_progress_bar = True,
    save_generated_images = False,
    display_images = True,
)

In [ ]:
logger.setLevel(logging.INFO) # return info

# Inference with SDXL

The tasks that can be used with SDXL:

In [ ]:
from stablepy import SDXL_TASKS

SDXL_TASKS


When switching between different tasks, you may encounter an out-of-memory (OOM) issue if there isn't enough GPU memory available, particularly with SDXL. To avoid this, you can set `retain_task_model_in_memory=False` in `model.load_pipe` or `Model_Diffusers` to save some VRAM. However, in many cases, this may not be sufficient, and you may need to restart the kernel or runtime in Colab to resolve the issue.

In [ ]:
model_name = "./Juggernaut-XL_v9_RunDiffusionPhoto_v2.safetensors"  # SDXL safetensors

model.load_pipe(
    base_model_id = model_name,
    task_name = "openpose",
    retain_task_model_in_cache=False,
)

We will perform OpenPose with the following image using SDXL.

In [ ]:
control_image_2 = download_image("https://upload.wikimedia.org/wikipedia/commons/f/f8/Model_Posing_On_Typical_Studio_Set.jpg")

In [ ]:
images, image_list = model(
    image = control_image_2,
    prompt = "a man with a pink jacket in the jungle",
    negative_prompt = "worst quality",

    # Relative resolution
    image_resolution = 1024,
    preprocessor_name = "Openpose", # for get the image preprocessor
    sampler="Euler a",
)

for image in images:
  display(image)

# IP Adapter

IP-Adapter enhances diffusion models by adding a dedicated image cross-attention layer for better image-specific feature learning and adaptability.

In [ ]:
from stablepy import IP_ADAPTERS_SD, IP_ADAPTERS_SDXL

IP_ADAPTERS_SDXL


You can specify a list of different IP adapter models with their respective images, but combinations between normal IP adapters and FaceID adapters are not enabled. Additionally, base_vit_G models cannot be combined with other models because they use a different image encoder. This image encoder is necessary for most IP adapter models and will occupy additional space in VRAM.


In [ ]:
img_ip = download_image("https://upload.wikimedia.org/wikipedia/commons/3/3f/TechCrunch_Disrupt_2019_%2848834434641%29_%28cropped%29.jpg")

In [ ]:
model.load_pipe(
    base_model_id = model_name,
    task_name = "txt2img",
    retain_task_model_in_cache=False,
)

In [ ]:
images, image_list = model(
    prompt = "a man with a pink jacket in the jungle",
    negative_prompt = "worst quality",

    img_width = 1024,
    img_height = 1024,
    num_images = 1,
    num_steps = 30,
    guidance_scale = 8.0,
    sampler="Euler a",

    ip_adapter_image = [img_ip],
    ip_adapter_mask = [],
    ip_adapter_model = ["plus_face"],
    ip_adapter_scale = [0.9],
    ip_adapter_mode = ["original"],

    display_images = True,
)

#### Multi IP-Adapter



In [ ]:
images, image_list = model(
    prompt = "a man with a pink jacket in the jungle",
    negative_prompt = "worst quality",

    img_width = 1024,
    img_height = 1024,
    num_images = 1,
    num_steps = 30,
    guidance_scale = 8.0,
    sampler="Euler a",

    ip_adapter_image = [img_ip, control_image],  # face image and the bird image
    ip_adapter_mask = [],                        # You can specify masks to isolate the influence area of each
    ip_adapter_model = ["plus_face", "base"],
    ip_adapter_scale = [0.9, 1.0],               # Scale 0.0 is equivalent to disabled
    ip_adapter_mode = ["original", "style"],     # The second IP adapter will take the style properties of the bird image

    display_images = True,
)

`ip_adapter_mode` specifies which layers are active in the IP adapter model, certain layers have specific influence on how features are extracted from the IP image, the valid options for this are "original", "style", "layout" and "style+layout"

# Diffusers format


You can also load models in the Diffusers format. This format divides the model into different parts, which allows you to load individual sections from various models more easily. For instance, models like SD 1.5 and SDXL can be loaded using the repository name as shown in this example: [RealVisXL_V2.0](https://huggingface.co/SG161222/RealVisXL_V2.0/tree/main). This repository contains folders corresponding to each section of the model such as unet, vae, text encoder, and more.

Another characteristic of the diffusers format is that it can use either the safetensors or bin extension. Currently, you can only use diffuser models in the safetensors extension because they offer better performance and are safer than bin files. To verify if a diffuser model is in safetensors format, check the [unet folder](https://huggingface.co/SG161222/RealVisXL_V2.0/tree/main/unet) and see if it ends with the safetensors extension.

In [ ]:
repo = "SG161222/RealVisXL_V2.0"

model.load_pipe(
    base_model_id = repo,
    task_name = "txt2img",
)

The T2I-Adapter depth is similar to that of ControlNet and uses less VRAM

In [ ]:
# Example sdxl_depth-midas
model.load_pipe(
    base_model_id = repo, # sdxl repo
    task_name = "sdxl_depth-midas_t2i",
    retain_task_model_in_cache=False,
)

# We can also use multiple styles in a list ["Silhouette", "Kirigami"]
images, image_list = model(
    image = control_image,
    prompt = "a green bird",
    negative_prompt = "worst quality",

    # If we want to use the preprocessor
    t2i_adapter_preprocessor = True,
    preprocess_resolution = 1024,

    # Relative resolution
    image_resolution = 1024,

    sampler="DPM++ 2M SDE Lu", # Specific variant for SDXL. We can also use euler at final with "DPM++ 2M SDE Ef"

    t2i_adapter_conditioning_scale = 1.0,
    t2i_adapter_conditioning_factor = 1.0,

    display_images = True,
)

# ControlNet pattern

It is used to generate images with a QR code but can also be used to generate optical patterns.

In [ ]:
spiral_image = download_image("https://upload.wikimedia.org/wikipedia/en/6/6c/Screwtop_spiral.jpg")

In [ ]:
model.load_pipe(
    base_model_id = repo,
    task_name = "pattern",
)

images, image_list = model(
    image = spiral_image,
    prompt = "a jungle landscape",
    negative_prompt = "worst quality",
    sampler="DPM++ 2M SDE Lu",
    image_resolution = 1024,
)

for image in images:
  display(image)

# Utils

In [ ]:
# Load beta styles
model.load_beta_styles()

In [ ]:
model.STYLE_NAMES

In [ ]:
# For more details about the parameters
help(model.__call__)

# Common issues and potential solutions:

- **Black image output**: This problem happens when a tensor reaches a certain value and "explodes". To prevent this, avoid using high weights with LoRAs or IP Adapter. FreeU can also worsen the issue. A common fix is to use more stable VAE. For SDXL, you don't need to specify a VAE because the most stable one `madebyollin/sdxl-vae-fp16-fix`, automatically loads for execution.

- **Distorted or very strange images**: This usually occurs due to prompt weight. In this implementation, the emphasis level set using Compel or Classic is particularly sensitive. It's best to use low prompt weights. Similarly, for LoRAs, it's recommended to use low scales. Also, using Classic variants like Classic-original can help; It has a normalization method to avoid extreme peaks that can greatly distort the outcome.

- **Pony Diffusion not producing good images**: Compatibility with the model can be tricky. However, you can try using sampler DPM++ 1s or DPM2 with Compel or Classic prompt weights to improve results.
